# Dimension Generation Tutorial

## Learning Objectives

By completing this tutorial, you will:
- ✅ Generate dimension tuples (combinations) using LLMs
- ✅ Convert tuples to natural language queries
- ✅ Validate and filter generated queries
- ✅ Optimize costs with parallel processing
- ✅ Export queries to CSV for systematic testing

## Prerequisites

- Completed [Error Analysis Concepts Tutorial](error_analysis_concepts.md)
- Identified 3-5 key dimensions for query generation
- OpenAI API key configured in `.env` file

## Estimated Time

**Execution Time:** 10-15 minutes (depending on API latency)

**⚠️ Cost Warning:** This notebook makes LLM API calls. Estimated cost: $0.10-0.30 for full execution (using gpt-4o-mini).

---

## Setup and Imports

In [ ]:
# Add project root to path for imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent.parent))

# Load environment variables
from dotenv import load_dotenv
import os
load_dotenv(Path.cwd().parent.parent / '.env')

# Verify API key is loaded
if not os.environ.get('OPENAI_API_KEY'):
    raise ValueError("OPENAI_API_KEY not found in environment. Check your .env file.")

print("✓ Environment configured")

In [ ]:
# Import required libraries
import json
from typing import List, Dict, Any
from pydantic import BaseModel
import pandas as pd
from litellm import completion
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm

print("✓ All libraries imported successfully")

---

## Why Dimension-Based Query Generation?

In **HW1**, you manually created 10+ diverse queries. This approach:
- ✅ Works for small test sets
- ❌ Doesn't scale to 100+ queries
- ❌ Hard to ensure balanced coverage
- ❌ Time-consuming and error-prone

**Dimension-based generation** solves this:
1. **Define dimensions** (cuisine, dietary, meal type, time, etc.)
2. **Generate tuples** - LLM creates diverse combinations
3. **Generate queries** - LLM converts tuples to natural language
4. **Validate** - Filter unrealistic or duplicate queries
5. **Export** - Save to CSV for systematic testing

**Key Benefits:**
- 🚀 Generate 50-100 queries in minutes
- 📊 Systematic coverage across all dimensions
- 🔄 Reproducible (can regenerate with different seeds)
- ✅ Includes natural language variations

**Reference:** See [`generate_synthetic_queries.py`](generate_synthetic_queries.py) for full implementation.

---

## Step 1: Define Your Dimensions

Based on **error analysis** (HW2 Part 2), identify dimensions where your bot frequently fails or needs comprehensive testing.

For our Recipe Bot, we'll use **6 dimensions**:

In [ ]:
# Define dimension values

DIMENSIONS = {
    "DietaryNeedsOrRestrictions": [
        "vegan", "vegetarian", "gluten-free", "dairy-free", "keto", "paleo",
        "halal", "kosher", "no restrictions", "pescatarian", "low-carb",
        "low-sodium", "nut-free", "egg-free", "soy-free", "FODMAP",
        "diabetic-friendly", "high-protein"
    ],
    
    "AvailableIngredientsFocus": [
        "must_use_specific: chicken, rice, vegetables",
        "must_use_specific: pasta, tomatoes, cheese",
        "must_use_specific: eggs, spinach, cheese",
        "must_use_specific: salmon, lemon, herbs",
        "general_pantry: basic ingredients",
        "no_specific_ingredients: open to suggestions"
    ],
    
    "CuisinePreference": [
        "specific_cuisine: Italian",
        "specific_cuisine: Mexican",
        "specific_cuisine: Thai",
        "specific_cuisine: Indian",
        "specific_cuisine: Chinese",
        "specific_cuisine: Mediterranean",
        "specific_cuisine: Japanese",
        "any_cuisine",
        "avoid_specific: spicy"
    ],
    
    "SkillLevelEffort": [
        "beginner_easy_low_effort",
        "intermediate_moderate_effort",
        "advanced_complex_high_effort"
    ],
    
    "TimeAvailability": [
        "quick_under_30_mins",
        "moderate_30_to_60_mins",
        "flexible_no_time_constraint"
    ],
    
    "QueryStyleAndDetail": [
        "short_keywords_minimal_detail",
        "natural_question_moderate_detail",
        "detailed_request_high_detail"
    ]
}

# Display dimension counts
for dim, values in DIMENSIONS.items():
    print(f"{dim}: {len(values)} values")

print(f"\nTotal possible combinations: {18 * 6 * 9 * 3 * 3 * 3:,}")
print("(We'll generate a diverse subset of 10-20 tuples)")

---

## Step 2: Define Pydantic Models

Pydantic provides **structured output** from LLM responses. This ensures we get valid, parseable data.

In [ ]:
# Pydantic models for structured LLM output

class DimensionTuple(BaseModel):
    """Represents one combination of dimension values."""
    DietaryNeedsOrRestrictions: str
    AvailableIngredientsFocus: str
    CuisinePreference: str
    SkillLevelEffort: str
    TimeAvailability: str
    QueryStyleAndDetail: str

class DimensionTuplesList(BaseModel):
    """List of dimension tuples returned by LLM."""
    tuples: List[DimensionTuple]

class QueriesList(BaseModel):
    """List of natural language queries."""
    queries: List[str]

class QueryWithDimensions(BaseModel):
    """Query with associated dimension tuple for tracking."""
    id: str
    query: str
    dimension_tuple: DimensionTuple
    is_realistic_and_kept: int = 1
    notes_for_filtering: str = ""

# Test the models
example_tuple = DimensionTuple(
    DietaryNeedsOrRestrictions="vegan",
    AvailableIngredientsFocus="general_pantry",
    CuisinePreference="specific_cuisine: Italian",
    SkillLevelEffort="beginner_easy_low_effort",
    TimeAvailability="quick_under_30_mins",
    QueryStyleAndDetail="natural_question_moderate_detail"
)

print("Example Dimension Tuple:")
print(example_tuple.model_dump_json(indent=2))

---

## Step 3: LLM Helper Function

In [ ]:
# Configuration
MODEL_NAME = "gpt-4o-mini"  # Cost-effective model for generation

def call_llm(messages: List[Dict[str, str]], response_format: Any) -> Any:
    """Call LLM with retry logic for robustness."""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = completion(
                model=MODEL_NAME,
                messages=messages,
                response_format=response_format
            )
            return response_format(**json.loads(response.choices[0].message.content))
        except Exception as e:
            if attempt == max_retries - 1:
                raise e
            time.sleep(1)  # Wait before retry
    
print("✓ LLM helper function defined")

---

## Step 4: Generate Dimension Tuples

We'll ask the LLM to generate **diverse combinations** of dimension values.

**Key prompt engineering principles:**
- Emphasize **balanced coverage** (don't over-represent any value)
- Provide **example tuples** showing realistic combinations
- Request **weird/unusual combinations** to test edge cases

In [ ]:
def generate_dimension_tuples(num_tuples: int = 10) -> List[DimensionTuple]:
    """Generate diverse dimension tuples using LLM."""
    
    prompt = f"""Generate {num_tuples} diverse combinations of dimension values for a recipe chatbot.
Each combination should represent a different user scenario. Ensure balanced coverage across all dimensions - don't over-represent any particular value or combination.

Important: Aim for an even distribution across all dimensions. For example:
- Don't generate too many dietary restrictions combinations
- Don't focus too heavily on quick recipes
- Don't over-represent any particular cuisine
- Vary the query styles naturally
- Try to use weird combinations of ingredients required in AvailableIngredientsFocus

DietaryNeedsOrRestrictions:
- vegan, vegetarian, gluten-free, dairy-free, keto, paleo, halal, kosher, no restrictions, pescatarian, low-carb, low-sodium, nut-free, egg-free, soy-free, FODMAP, diabetic-friendly, high-protein

AvailableIngredientsFocus:
- must_use_specific: [list of ingredients]
- general_pantry: basic ingredients
- no_specific_ingredients: open to suggestions

CuisinePreference:
- specific_cuisine: [cuisine type]
- any_cuisine
- avoid_specific: [cuisine type]

SkillLevelEffort:
- beginner_easy_low_effort
- intermediate_moderate_effort
- advanced_complex_high_effort

TimeAvailability:
- quick_under_30_mins
- moderate_30_to_60_mins
- flexible_no_time_constraint

QueryStyleAndDetail:
- short_keywords_minimal_detail
- natural_question_moderate_detail
- detailed_request_high_detail

Generate {num_tuples} unique dimension tuples with balanced diversity across all dimensions."""
    
    messages = [{"role": "user", "content": prompt}]
    
    response = call_llm(messages, DimensionTuplesList)
    return response.tuples

print("✓ Tuple generation function defined")

In [ ]:
# Generate tuples
print("Generating 10 dimension tuples...")
dimension_tuples = generate_dimension_tuples(num_tuples=10)

print(f"\n✓ Generated {len(dimension_tuples)} tuples\n")

# Display first 3 tuples
for i, tuple_obj in enumerate(dimension_tuples[:3], 1):
    print(f"Tuple {i}:")
    print(json.dumps(tuple_obj.model_dump(), indent=2))
    print()

---

## Step 5: Generate Natural Language Queries

Now we convert each dimension tuple into a **realistic natural language query** that a user might ask.

**Key features:**
- Multiple queries per tuple (to increase coverage)
- Natural language variations (typos, lowercase, emojis, text speak)
- Realistic user query styles

In [ ]:
def generate_queries_for_tuple(dimension_tuple: DimensionTuple, num_queries: int = 3) -> List[str]:
    """Generate natural language queries for a given dimension tuple."""
    
    prompt = f"""Generate {num_queries} different natural language queries for a recipe chatbot based on these characteristics:
{dimension_tuple.model_dump_json(indent=2)}

The queries should:
1. Sound like real users asking for recipe help
2. Naturally incorporate all the dimension values
3. Vary in style and detail level
4. Be realistic and practical
5. Include natural variations in typing style, such as:
   - Some queries in all lowercase
   - Some with random capitalization
   - Some with common typos
   - Some with missing punctuation
   - Some with emojis or text speak

Generate {num_queries} unique queries that match the given dimensions, varying the text style naturally."""
    
    messages = [{"role": "user", "content": prompt}]
    
    response = call_llm(messages, QueriesList)
    return response.queries

print("✓ Query generation function defined")

In [ ]:
# Test query generation on first tuple
print("Generating 3 queries for the first dimension tuple...\n")

test_queries = generate_queries_for_tuple(dimension_tuples[0], num_queries=3)

print(f"Dimension Tuple:")
print(json.dumps(dimension_tuples[0].model_dump(), indent=2))
print(f"\nGenerated Queries:")
for i, query in enumerate(test_queries, 1):
    print(f"{i}. {query}")

---

## Step 6: Parallel Processing for Efficiency

Generating queries sequentially is slow. We'll use **ThreadPoolExecutor** to parallelize LLM calls.

**Benefits:**
- 🚀 5-10x faster generation
- 📊 Progress tracking with tqdm
- 🔄 Handles failures gracefully

In [ ]:
def generate_queries_parallel(
    dimension_tuples: List[DimensionTuple],
    queries_per_tuple: int = 3,
    max_workers: int = 5
) -> List[QueryWithDimensions]:
    """Generate queries in parallel for all dimension tuples."""
    
    all_queries = []
    query_id = 1
    
    print(f"Generating {queries_per_tuple} queries each for {len(dimension_tuples)} dimension tuples...")
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_tuple = {
            executor.submit(generate_queries_for_tuple, dim_tuple, queries_per_tuple): i 
            for i, dim_tuple in enumerate(dimension_tuples)
        }
        
        # Process completed tasks with progress bar
        with tqdm(total=len(dimension_tuples), desc="Generating Queries") as pbar:
            for future in as_completed(future_to_tuple):
                tuple_idx = future_to_tuple[future]
                try:
                    queries = future.result()
                    for query in queries:
                        all_queries.append(QueryWithDimensions(
                            id=f"SYN{query_id:03d}",
                            query=query,
                            dimension_tuple=dimension_tuples[tuple_idx]
                        ))
                        query_id += 1
                    pbar.update(1)
                except Exception as e:
                    print(f"\nTuple {tuple_idx + 1} failed: {e}")
                    pbar.update(1)
    
    return all_queries

print("✓ Parallel generation function defined")

In [ ]:
# Generate all queries in parallel
queries = generate_queries_parallel(
    dimension_tuples=dimension_tuples,
    queries_per_tuple=3,
    max_workers=5
)

print(f"\n✓ Generated {len(queries)} total queries from {len(dimension_tuples)} tuples")

---

## Step 7: Quality Validation

Before exporting, check for:
- **Duplicates** - Remove exact duplicate queries
- **Too short** - Filter queries <5 characters
- **Too generic** - Flag vague queries like "recipe"
- **Diversity** - Ensure variety across dimensions

In [ ]:
# Check for duplicates
query_texts = [q.query for q in queries]
unique_queries = set(query_texts)

print(f"Total queries: {len(query_texts)}")
print(f"Unique queries: {len(unique_queries)}")
print(f"Duplicates: {len(query_texts) - len(unique_queries)}")

# Check query lengths
query_lengths = [len(q.query) for q in queries]
print(f"\nQuery length statistics:")
print(f"  Min: {min(query_lengths)} characters")
print(f"  Max: {max(query_lengths)} characters")
print(f"  Average: {sum(query_lengths) / len(query_lengths):.1f} characters")

# Display sample queries
print(f"\nSample queries:")
for i, q in enumerate(queries[:5], 1):
    print(f"{i}. [{q.id}] {q.query}")

---

## Step 8: Export to CSV

Save queries to CSV for use with [`scripts/bulk_test.py`](../../scripts/bulk_test.py).

In [ ]:
def save_queries_to_csv(queries: List[QueryWithDimensions], filename: str = "synthetic_queries_for_analysis.csv"):
    """Save generated queries to CSV."""
    
    output_path = Path.cwd() / filename
    
    # Convert to DataFrame
    df = pd.DataFrame([
        {
            'id': q.id,
            'query': q.query,
            'dimension_tuple_json': q.dimension_tuple.model_dump_json(),
            'is_realistic_and_kept': q.is_realistic_and_kept,
            'notes_for_filtering': q.notes_for_filtering
        }
        for q in queries
    ])
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✓ Saved {len(queries)} queries to {output_path}")
    
    return df

# Save queries
df = save_queries_to_csv(queries)

In [ ]:
# Display first few rows
print("\nFirst 5 rows of exported CSV:\n")
print(df[['id', 'query']].head())

---

## Cost Estimation and Optimization

### Estimated Costs (using gpt-4o-mini)

- **Input tokens:** ~500 tokens per tuple generation + ~300 per query generation
- **Output tokens:** ~200 tokens per tuple generation + ~50 per query
- **Total:** For 10 tuples × 3 queries = ~30,000 tokens
- **Cost:** ~$0.10-0.30 (gpt-4o-mini rates)

### Optimization Tips

1. **Use cheaper models** - gpt-4o-mini instead of gpt-4o
2. **Cache results** - Save generated queries, don't regenerate
3. **Reduce queries_per_tuple** - 1-2 instead of 3-5
4. **Batch processing** - Generate more tuples at once (fewer API calls)
5. **Use local models** - Llama, Mistral (free, but slower/lower quality)

---

## Next Steps

After generating queries:

1. **Run bulk test** on generated queries:
   ```bash
   python scripts/bulk_test.py --queries homeworks/hw2/synthetic_queries_for_analysis.csv
   ```

2. **Perform open coding** on bot responses (see [Error Analysis Concepts](error_analysis_concepts.md))

3. **Build failure taxonomy** (see [Failure Mode Taxonomy Tutorial](failure_mode_taxonomy_tutorial.md))

4. **Iterate:** Generate more queries targeting specific failure modes

---

## Key Takeaways

- ✅ **LLM-based generation scales** - Create 100+ queries in minutes
- ✅ **Dimensions ensure coverage** - Systematic testing across all variations
- ✅ **Parallel processing saves time** - 5-10x faster with ThreadPoolExecutor
- ✅ **Structured output with Pydantic** - Reliable, parseable responses
- ✅ **Natural language variations** - Typos, lowercase, emojis make queries realistic
- ✅ **Cost-effective** - gpt-4o-mini makes this affordable at scale

---

## Further Reading

- [Error Analysis Concepts](error_analysis_concepts.md) - Open and axial coding methodology
- [Failure Mode Taxonomy Tutorial](failure_mode_taxonomy_tutorial.md) - Build structured failure taxonomies
- [generate_synthetic_queries.py](generate_synthetic_queries.py) - Full script implementation
- [HW2 README](README.md) - Complete assignment instructions

---

**Tutorial Status:** ✅ Complete
**Last Updated:** 2025-10-29
**Maintainer:** AI Evaluation Course Team